In [2]:
#https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb

!pip install torchtext==0.6.0
# BLEU Score 계산을 위한 라이브러리 업데이트

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:0000:01


In [4]:
# 데이터 전처리
# spaCy 라이브러리: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
# 영어(Engilsh)와 독일어(Deutsch) 전처리 모듈 설치
# 가상환경 terminal에서 설치 

!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

/Users/kmsviola_pro16/opt/anaconda3/envs/nlp_37/bin/python: No module named spacy
/Users/kmsviola_pro16/opt/anaconda3/envs/nlp_37/bin/python: No module named spacy


In [6]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # English tokenization
spacy_de = spacy.load('de_core_news_sm') # German tokenization

In [7]:
#간단한 토큰화 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"index {i}: {token.text}")

index 0: I
index 1: am
index 2: a
index 3: graduate
index 4: student
index 5: .


In [8]:
# 문장에 대해 토큰화를 수행하여 리스트 형태로 반환하기 
# 독일어 문장을 토큰화 하는 함수 (순서를 뒤집지 않음) 
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [9]:
# 필드 라이브러리를 이용해 데이터셋에 대해 구체적인 전처리 내용을 명시합니다. 
# 어떠한 데이터셋에 대해 전처리를 세팅
# Seq2Seq 모델과는 다르게 batch_first 속성 값을 True로 설정합니다. 
# 번역 목표 
 # 소스(SRC) : 독일어
 # 목표(TRG) : 영어 

In [10]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

# 시작은 <sos> 끝은 <eos>를 붙이고 lower = True는 소문자로 변환, transformer에 입력을 넣을 때는 텐서의 차원에서 시퀀스보다는 배치가 먼저 오도록 만드는 경우가 많다. 

In [11]:
# 대표적인 영어-독어 번역 데이터셋인 Multi30k를 불러오기 독일어를 영어로 바꾸는 태스크..
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

In [12]:
print(f"training_dataset_size: {len(train_dataset.examples)}")
print(f"valid_dataset_size: {len(valid_dataset.examples)}")
print(f"test_dataset_size: {len(test_dataset.examples)}")

training_dataset_size: 29000
valid_dataset_size: 1014
test_dataset_size: 1000


In [13]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30]))
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

{'src': ['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.'], 'trg': ['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']}
['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [14]:
# 단어 사전 만들기
# 필드 (field) 객체 build_vocab메서드를 이용해 영어와 독어의 단어 사전을 생성
# 최소 2번 이상 등장한 단어만을 선택

SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7853
len(TRG): 5893


In [15]:
print(TRG.vocab.stoi["abcabc"]) #없는 단어 0
print(TRG.vocab.stoi[TRG.pad_token]) #패딩 (padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos> : 2
print(TRG.vocab.stoi["<eos>"]) # <eos> : 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


In [17]:
# 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 합니다. 
# 따라서, 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다. 
# 이를 위해 BucketIterator를 사용합니다. 
# 배치크기 (batch size): 128

In [16]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset), batch_size = BATCH_SIZE, device = device
)

In [15]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"first batch size: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[1]):
        print(f"index {i}: {src[0][i].item()}")  ## 여기에서는 [Seq_num, Seq_len]
        # .item() : tensor에 저장된 값만을 가져옴
    # 첫번째 배치만 확인   
    break

first batch size: torch.Size([128, 36])
index 0: 2
index 1: 5
index 2: 13
index 3: 7
index 4: 206
index 5: 475
index 6: 0
index 7: 9
index 8: 23
index 9: 590
index 10: 14
index 11: 2511
index 12: 7
index 13: 33
index 14: 3003
index 15: 28
index 16: 6007
index 17: 4
index 18: 3
index 19: 1
index 20: 1
index 21: 1
index 22: 1
index 23: 1
index 24: 1
index 25: 1
index 26: 1
index 27: 1
index 28: 1
index 29: 1
index 30: 1
index 31: 1
index 32: 1
index 33: 1
index 34: 1
index 35: 1


In [16]:
## Multi head Attention
# attention 은 세가지 요소를 입력으로 받습니다. 
# 쿼리, 키, 값 - 현재 구현에서는 Q, K, V의 차원이 모두 같습니다. 
# 하이퍼 파라미터
# -> hidden_dim : 하나의 단어에 대한 임베딩 차원, n_heads : 헤드의 수 = scaled dot=product attention의 개수
# -> dropout_ratio : 드롭아웃 비율

In [ ]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        assert hidden_dim % n_heads == 0
        
        self.hidden_dim = hidden_dim # 임베딩차원
        self.n_heads = n_heads # 헤드의 개수 : 서로 다른 attention 컨셉의 수 
        self.head_dim = hidden_dim//n_heads # 각 헤드에서의 임베딩의 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어 

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio) 
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

        
